# IMDB MySQL Database Part Two
* James Belk
* 3/28/2023

## Imports

In [1]:
# Standard Imports
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Additional Imports
import os, json, math, time
import tmdbsimple as tmdb
from tqdm.notebook import tqdm_notebook

In [2]:
FOLDER = 'Data/'
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['.ipynb_checkpoints',
 'final_akas.csv.gz',
 'final_basics.csv.gz',
 'final_ratings.csv.gz',
 'title_akas.csv.gz',
 'title_basics.csv.gz',
 'title_ratings.csv.gz']

## API

In [3]:
import json
with open('/Users/james/.secret/tmdb_api.json', 'r') as f:
    login = json.load(f)
## Display the keys of the loaded dict
login.keys()

tmdb.API_KEY =  login['api-key']

## Functions

In [4]:
def get_movie_with_rating(movie_id):
    # Get the movie object for the current id
    movie = tmdb.Movies(movie_id)
    # save the .info .releases dictionaries
    info = movie.info()
    
    releases = movie.releases()
    # Loop through countries in releases
    for c in releases['countries']:
        #if the country abbreviation==US
        if c['iso_3166_1']=='US':
            # save a 'certification' key in info with the certification
            info['certification'] = c['certification']
            
    return info

def write_json(new_data, filename): 
    """Appends a list of records (new_data) to a json file (filename). 
    Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""  
    
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)

## Dataset

In [5]:
basics = pd.read_csv('Data/final_basics.csv.gz')

In [6]:
movie_ids = basics['tconst'].copy
movie_ids

<bound method NDFrame.copy of 0       tt0035423
1       tt0113026
2       tt0113092
3       tt0114447
4       tt0115937
          ...    
2989    tt9212730
2990    tt9228234
2991    tt9412476
2992    tt9555974
2993    tt9578462
Name: tconst, Length: 2994, dtype: object>

## Create Required Lists for the Loop

In [7]:
YEARS_TO_GET = [2000, 2001, 2022]
errors = []

In [8]:
# Start of the outer loop
for YEAR in tqdm_notebook(YEARS_TO_GET, desc='YEARS', position=0):
    # Defining the JSON file to store results for year
    JSON_FILE = f'{FOLDER}tqdb_api_results_{YEAR}.json'
    # chec kif file exists
    file_exists = os.path.isfile(JSON_FILE)
    # If it does not exist: create it
    if file_exists == False:
    # save an empty dict with just 'imdb_id' to the new json file
        with open(JSON_FILE, 'w') as f:
            json.dump([{'imdb_id':0}], f)
            # Saving new year
    df = basics.loc[basics['startYear']==YEAR].copy()
    # Saving movie ids to list
    movie_ids = df['tconst'].copy()
    # Load existing data from json into a dataframe called 'previous_df'
    previous_df = pd.read_json(JSON_FILE)
    # filter out any ids that are already in the JSON_FILE
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]

    #Get index and movie id from list
    # INNER Loop
    for movie_id in tqdm_notebook(movie_ids_to_get,
                          desc=f'Movies from {YEAR}',
                          position=1,
                          leave=True):
        try:
        # Retrieve then data for the movie id
            temp = get_movie_with_rating(movie_id)  
        # Append/extend results to existing file using a pre-made function
            write_json(temp,JSON_FILE)
        # Short 20 ms sleep to prevent overwhelming server
            time.sleep(0.02)

        except Exception as e:
            errors.append([movie_id, e])

    final_year_df = pd.read_json(JSON_FILE)
    final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz", compression="gzip", index=False)

    print(f'- Total errors: {len(errors)}')

YEARS:   0%|          | 0/3 [00:00<?, ?it/s]

Movies from 2000:   0%|          | 0/1438 [00:00<?, ?it/s]

- Total errors: 210


Movies from 2001:   0%|          | 0/1556 [00:00<?, ?it/s]

- Total errors: 451


Movies from 2022: 0it [00:00, ?it/s]

- Total errors: 451


In [ ]:
test = get_movie_with_rating('tt0848228')
test

In [ ]:
test_ids = ["tt0848228", "tt0332280"]
results = []
errors = []
for movie_id in test_ids:
    try:
        movie_info = get_movie_with_rating(movie_id)
        results.append(movie_info)
    except Exception as e: 
        errors.append([movie_id, e])
    
    
display(pd.DataFrame(results))
print(f'- Number of errors: {len(errors)}')
errors